![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Legal Deidentification

In [0]:
from johnsnowlabs import * 

# Deidentification Model

Some legal information can be considered sensitive. (e.g.,document, organization, address, signer)

In [0]:
documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = nlp.SentenceDetectorDLModel.pretrained("sentence_detector_dl","xx")\
    .setInputCols(["document"])\
    .setOutputCol("sentence")
    #.setCustomBounds(["\n\n"])

tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

embeddings = nlp.RoBertaEmbeddings.pretrained("roberta_embeddings_legal_roberta_base","en") \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("embeddings")

legal_ner = legal.NerModel.pretrained("legner_contract_doc_parties", "en", "legal/models")\
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner") 
    #.setLabelCasing("upper")

ner_converter = legal.NerConverterInternal() \
    .setInputCols(["sentence", "token", "ner"])\
    .setOutputCol("ner_chunk")\
    .setReplaceLabels({"ALIAS": "PARTY"}) # "ALIAS" are secondary names of companies, so let's extract them also as PARTY

nlpPipeline = nlp.Pipeline(stages=[
      documentAssembler, 
      sentenceDetector,
      tokenizer,
      embeddings,
      legal_ner,
      ner_converter])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[ | ][OK!]
roberta_embeddings_legal_roberta_base download started this may take some time.
Approximate size to download 447.2 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]
legner_contract_doc_parties download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][OK!]

### Pretrained NER models extracts:
- Document
- Date
- Party (Organization Name)
- Alias

In [0]:
legal_ner.getClasses()

Out[3]: ['O',
 'I-DOC',
 'B-EFFDATE',
 'B-ALIAS',
 'I-ALIAS',
 'B-PARTY',
 'I-EFFDATE',
 'I-PARTY',
 'B-DOC']

In [0]:
text = """THIS STRATEGIC ALLIANCE AGREEMENT ("Agreement") is made and entered into as of December 14, 2016 , by and between Hyatt Franchising Latin America, L.L.C., a limited liability company organized and existing under the laws of the State of Delaware """

In [0]:
result = model.transform(spark.createDataFrame([[text]]).toDF("text"))

In [0]:
from pyspark.sql import functions as F

result_df = result.select(F.explode(F.arrays_zip(result.token.result, 
                                                 result.ner.result)).alias("cols")) \
                  .select(F.expr("cols['0']").alias("token"),
                          F.expr("cols['1']").alias("ner_label"))

In [0]:
result_df.select("token", "ner_label").groupBy('ner_label').count().orderBy('count', ascending=False).show(truncate=False)

+---------+-----+
ner_label|count|
+---------+-----+
O |31 |
I-PARTY |5 |
I-EFFDATE|3 |
I-DOC |2 |
B-DOC |1 |
B-PARTY |1 |
B-EFFDATE|1 |
+---------+-----+

### Check extracted sensitive entities

In [0]:
result.select(F.explode(F.arrays_zip(result.ner_chunk.result, 
                                     result.ner_chunk.metadata)).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+--------------------------------------+---------+
chunk |ner_label|
+--------------------------------------+---------+
STRATEGIC ALLIANCE AGREEMENT |DOC |
December 14, 2016 |EFFDATE |
Hyatt Franchising Latin America, L.L.C|PARTY |
+--------------------------------------+---------+

## Masking and Obfuscation

### Replace these enitites with Tags

In [0]:
ner_converter = legal.NerConverterInternal()\
      .setInputCols(["sentence", "token", "ner"])\
      .setOutputCol("ner_chunk") 

deidentification = legal.DeIdentification() \
      .setInputCols(["sentence", "token", "ner_chunk"]) \
      .setOutputCol("deidentified") \
      .setMode("mask")\
      .setReturnEntityMappings(True) #  return a new column to save the mappings between the mask/obfuscated entities and original entities.
      #.setMappingsColumn("MappingCol") # change the name of the column, 'aux' is default

deidPipeline = nlp.Pipeline(stages=[
      documentAssembler, 
      sentenceDetector,
      tokenizer,
      embeddings,
      legal_ner,
      ner_converter,
      deidentification])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model_deid = deidPipeline.fit(empty_data)

In [0]:
result = model_deid.transform(spark.createDataFrame([[text]]).toDF("text"))

In [0]:
result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| embeddings| ner| ner_chunk| deidentified| aux|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
THIS STRATEGIC AL...|[{document, 0, 24...|[{document, 0, 24...|[{token, 0, 3, TH...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 5, 32, S...|[{document, 0, 18...|[{chunk, 5, 9, <D...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

In [0]:
result.select(F.explode(F.arrays_zip(result.sentence.result, result.deidentified.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("sentence"), F.expr("cols['1']").alias("deidentified")).toPandas()

Out[12]:

,sentence,deidentified
0,"THIS STRATEGIC ALLIANCE AGREEMENT (""Agreement""...","THIS <DOC> (""Agreement"") is made and entered i..."


We have three modes to mask the entities in the Deidentification annotator. You can select the modes using the `.setMaskingPolicy()` parameter. The methods are the followings:

**“entity_labels”**: Mask with the entity type of that chunk. (default) <br/>
**“same_length_chars”**: Mask the deid entities with same length of asterix ( * ) with brackets ( [ , ] ) on both end. <br/>
**“fixed_length_chars”**: Mask the deid entities with a fixed length of asterix ( * ). The length is setting up using the `setFixedMaskLength()` method. <br/>

Let's try each of these and compare the results:

In [0]:
#deid model with "entity_labels"
deid_entity_labels= legal.DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk"])\
    .setOutputCol("deid_entity_label")\
    .setMode("mask")\
    .setReturnEntityMappings(True)\
    .setMaskingPolicy("entity_labels")

#deid model with "same_length_chars"
deid_same_length= legal.DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk"])\
    .setOutputCol("deid_same_length")\
    .setMode("mask")\
    .setReturnEntityMappings(True)\
    .setMaskingPolicy("same_length_chars")

#deid model with "fixed_length_chars"
deid_fixed_length= legal.DeIdentification()\
    .setInputCols(["sentence", "token", "ner_chunk"])\
    .setOutputCol("deid_fixed_length")\
    .setMode("mask")\
    .setReturnEntityMappings(True)\
    .setMaskingPolicy("fixed_length_chars")\
    .setFixedMaskLength(4)


deidPipeline = nlp.Pipeline(stages=[
      documentAssembler, 
      sentenceDetector,
      tokenizer,
      embeddings,
      legal_ner,
      ner_converter,
      deid_entity_labels,
      deid_same_length,
      deid_fixed_length])


empty_data = spark.createDataFrame([[""]]).toDF("text")
model_deid = deidPipeline.fit(empty_data)

In [0]:
policy_result = model_deid.transform(spark.createDataFrame([[text]]).toDF("text"))

In [0]:
policy_result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| embeddings| ner| ner_chunk| deid_entity_label| aux| deid_same_length| deid_fixed_length|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
THIS STRATEGIC AL...|[{document, 0, 24...|[{document, 0, 24...|[{token, 0, 3, TH...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 5, 32, S...|[{document, 0, 18...|[{chunk, 5, 8, **...|[{document, 0, 24...|[{document, 0, 17...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

In [0]:
policy_result.select(F.explode(F.arrays_zip(policy_result.sentence.result, 
                                            policy_result.deid_entity_label.result, 
                                            policy_result.deid_same_length.result, 
                                            policy_result.deid_fixed_length.result)).alias("cols")) \
             .select(F.expr("cols['0']").alias("sentence"),
                     F.expr("cols['1']").alias("deid_entity_label"),
                     F.expr("cols['2']").alias("deid_same_length"),
                     F.expr("cols['3']").alias("deid_fixed_length")).toPandas()

Out[16]:

,sentence,deid_entity_label,deid_same_length,deid_fixed_length
0,"THIS STRATEGIC ALLIANCE AGREEMENT (""Agreement""...","THIS <DOC> (""Agreement"") is made and entered i...","THIS [**************************] (""Agreement""...","THIS **** (""Agreement"") is made and entered in..."


### Mapping Column

In [0]:
result.select("aux").show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
aux |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
[{chunk, 5, 9, <DOC>, {originalChunk -> STRATEGIC ALLIANCE AGREEMENT, chunk -> 0, beginOriginalChunk -> 5, confidence -> 0.98066664, ner_source -> ner_chunk, entity -> DOC, endOriginalChunk -> 32, sentence -> 0}, []}, {chunk, 56, 64, <EFFDATE>, {originalChunk -> December 14, 2016, chunk -> 1, beginOriginalChunk -> 79, confidence -> 0.927725, ner_source -> ner_chunk, entity -> EFFDATE, endOriginalChunk -> 95, sentence -> 0}, []}, {chunk, 83, 89, <PARTY>, {originalChunk -> Hyatt Franchising Latin America, L.L.C, chunk -> 2, beginOriginalChunk -> 114, confidence -> 0.96851665, ner_source -> ner_chunk, entity -> PARTY, endOriginalChunk -> 151, sentence -> 0}, []}]|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

In [0]:
result.select(F.explode(F.arrays_zip(result.aux.metadata, 
                                     result.aux.result, 
                                     result.aux.begin, 
                                     result.aux.end)).alias("cols")) \
      .select(F.expr("cols['0']['originalChunk']").alias("chunk"),
              F.expr("cols['0']['beginOriginalChunk']").alias("beginChunk"),
              F.expr("cols['0']['endOriginalChunk']").alias("endChunk"),
              F.expr("cols['1']").alias("label"),
              F.expr("cols['2']").alias("beginLabel"),
              F.expr("cols['3']").alias("endLabel")).show(truncate=False)

+--------------------------------------+----------+--------+---------+----------+--------+
chunk |beginChunk|endChunk|label |beginLabel|endLabel|
+--------------------------------------+----------+--------+---------+----------+--------+
STRATEGIC ALLIANCE AGREEMENT |5 |32 |<DOC> |5 |9 |
December 14, 2016 |79 |95 |<EFFDATE>|56 |64 |
Hyatt Franchising Latin America, L.L.C|114 |151 |<PARTY> |83 |89 |
+--------------------------------------+----------+--------+---------+----------+--------+

## Reidentification

We can use `ReIdentification` annotator to go back to the original sentence.

In [0]:
reIdentification = legal.ReIdentification()\
    .setInputCols(["aux","deidentified"])\
    .setOutputCol("original")

In [0]:
reid_result = reIdentification.transform(result)

In [0]:
reid_result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| embeddings| ner| ner_chunk| deidentified| aux| original|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
THIS STRATEGIC AL...|[{document, 0, 24...|[{document, 0, 24...|[{token, 0, 3, TH...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 5, 32, S...|[{document, 0, 18...|[{chunk, 5, 9, <D...|[{document, 0, 24...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+

In [0]:
print(text)

reid_result.select('original.result').show(truncate=False)

THIS STRATEGIC ALLIANCE AGREEMENT ("Agreement") is made and entered into as of December 14, 2016 , by and between Hyatt Franchising Latin America, L.L.C., a limited liability company organized and existing under the laws of the State of Delaware 
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
result |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
[THIS STRATEGIC ALLIANCE AGREEMENT ("Agreement") is made and entered into as of December 14, 2016 , by and between Hyatt Franchising Latin America, L.L.C., a limited liability company organized and existing under the laws of the State of Delaware]|
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+

## Using multiple NER in the same pipeline

In [0]:
documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = nlp.SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

embeddings = nlp.RoBertaEmbeddings.pretrained("roberta_embeddings_legal_roberta_base","en") \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("embeddings")

legal_ner = legal.NerModel.pretrained("legner_contract_doc_parties", "en", "legal/models")\
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner") 
    #.setLabelCasing("upper")

ner_converter = legal.NerConverterInternal() \
    .setInputCols(["sentence", "token", "ner"])\
    .setOutputCol("ner_chunk")\
    .setReplaceLabels({"ALIAS": "PARTY"})

ner_signers = legal.NerModel.pretrained("legner_signers", "en", "legal/models")\
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner_signers") 
    #.setLabelCasing("upper")

ner_converter_signers = nlp.NerConverter() \
    .setInputCols(["sentence", "token", "ner_signers"]) \
    .setOutputCol("ner_signer_chunk")

chunk_merge = legal.ChunkMergeApproach()\
    .setInputCols("ner_signer_chunk", "ner_chunk")\
    .setOutputCol("deid_merged_chunk")

deidentification = legal.DeIdentification() \
    .setInputCols(["sentence", "token", "deid_merged_chunk"]) \
    .setOutputCol("deidentified") \
    .setMode("mask")\
    .setIgnoreRegex(True)


nlpPipeline = nlp.Pipeline(stages=[
      documentAssembler, 
      sentenceDetector,
      tokenizer,
      embeddings,
      legal_ner,
      ner_converter,
      ner_signers,
      ner_converter_signers,
      chunk_merge,
      deidentification])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

roberta_embeddings_legal_roberta_base download started this may take some time.
Approximate size to download 447.2 MB
[ | ][OK!]
legner_contract_doc_parties download started this may take some time.
[ | ][OK!]
legner_signers download started this may take some time.
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]

In [0]:
text = """ENTIRE AGREEMENT.  This Agreement contains the entire understanding of the parties hereto with respect to the transactions and matters contemplated hereby, supersedes all previous Agreements between i-Escrow and 2TheMart concerning the subject matter.

2THEMART.COM, INC.:                         I-ESCROW, INC.:

By:Dominic J. Magliarditi                By:Sanjay Bajaj Name: Dominic J. Magliarditi                Name: Sanjay Bajaj Title: President                            Title: VP Business Development Date: 6/21/99                               Date: 6/11/99 """ 

In [0]:
result = model.transform(spark.createDataFrame([[text]]).toDF("text"))

# legal_ner
result.select(F.explode(F.arrays_zip(result.ner_chunk.result, 
                                     result.ner_chunk.metadata)).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+----------------+---------+
chunk |ner_label|
+----------------+---------+
ENTIRE AGREEMENT|DOC |
+----------------+---------+

In [0]:
result = model.transform(spark.createDataFrame([[text]]).toDF("text"))

# ner_signers
result.select(F.explode(F.arrays_zip(result.ner_signer_chunk.result, 
                                     result.ner_signer_chunk.metadata)).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+-----------------------+--------------+
chunk |ner_label |
+-----------------------+--------------+
INC |PARTY |
J. Magliarditi |SIGNING_PERSON|
Bajaj |SIGNING_PERSON|
Dominic J. Magliarditi |SIGNING_PERSON|
Sanjay Bajaj |SIGNING_PERSON|
President |SIGNING_TITLE |
VP Business Development|SIGNING_TITLE |
+-----------------------+--------------+

In [0]:
result = model.transform(spark.createDataFrame([[text]]).toDF("text"))

# merged_chunk
result.select(F.explode(F.arrays_zip(result.deid_merged_chunk.result, 
                                     result.deid_merged_chunk.metadata)).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(truncate=False)

+-----------------------+--------------+
chunk |ner_label |
+-----------------------+--------------+
ENTIRE AGREEMENT |DOC |
INC |PARTY |
J. Magliarditi |SIGNING_PERSON|
Bajaj |SIGNING_PERSON|
Dominic J. Magliarditi |SIGNING_PERSON|
Sanjay Bajaj |SIGNING_PERSON|
President |SIGNING_TITLE |
VP Business Development|SIGNING_TITLE |
+-----------------------+--------------+

In [0]:
result.select(F.explode(F.arrays_zip(result.sentence.result, 
                                     result.deidentified.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("sentence"),
              F.expr("cols['1']").alias("deidentified")).toPandas()

Out[28]:

,sentence,deidentified
0,ENTIRE AGREEMENT.,<DOC>.
1,This Agreement contains the entire understandi...,This Agreement contains the entire understandi...
2,"2THEMART.COM, INC.: I-...","2THEMART.COM, INC.: I-..."


## Obfuscation mode

In the obfuscation mode **DeIdentificationModel** will replace sensitive entities with random values of the same type.

In [0]:
# This is the obfuscation dict for the new entities
obs_lines = """CTO#SIGNING_TITLE
Project Manager#SIGNING_TITLE
Sales Manager#SIGNING_TITLE
Business Manager#SIGNING_TITLE
Coordinator#SIGNING_TITLE
Officer#SIGNING_TITLE
Legal Agreement#DOC
Contract#DOC
Estate Document#DOC
official Document#DOC
Deed of Covenant#DOC
TURER INC#PARTY
Clark llc.#PARTY
SESA CO.#PARTY
John Snow Labs Inc#PARTY
MGT Trust Company, LLC.#PARTY
JAMES TURNER#SIGNING_PERSON
Juan Garcia#SIGNING_PERSON
Benjamin Dean#SIGNING_PERSON
Tommy Lee#SIGNING_PERSON
Dorothy Keen#SIGNING_PERSON
("AGREEMENT")#ALIAS
("TRADE COMPANY")#ALIAS
(the" Agreement")#ALIAS
("private company")#ALIAS
(the "Contract")#ALIAS
26-06-1990#EFFDATE
03/08/2025#EFFDATE
01/01/2045#EFFDATE
11/7/2016#EFFDATE
12-12-2022#EFFDATE """

with open ('obfuscate.txt', 'w') as f:
    f.write(obs_lines)

In [0]:
ner_converter_signers = nlp.NerConverter() \
    .setInputCols(["sentence", "token", "ner_signers"]) \
    .setOutputCol("ner_signer_chunk")

chunk_merge = legal.ChunkMergeApproach()\
    .setInputCols("ner_signer_chunk", "ner_chunk")\
    .setOutputCol("deid_merged_chunk")

obfuscation = legal.DeIdentification()\
    .setInputCols(["sentence", "token", "ner_signer_chunk"]) \
    .setOutputCol("deidentified") \
    .setMode("obfuscate")\
    .setObfuscateDate(True)\
    .setObfuscateRefFile('obfuscate.txt')\
    .setObfuscateRefSource("both") #default: "faker"


nlpPipeline = nlp.Pipeline(stages=[
      documentAssembler, 
      sentenceDetector,
      tokenizer,
      embeddings,
      legal_ner,
      ner_converter,
      ner_signers,
      ner_converter_signers,
      chunk_merge,
      obfuscation])

obfuscation_model = nlpPipeline.fit(empty_data)

In [0]:
text = """"Newegg" "Allied" Newegg Inc. Allied Esports International, Inc. By  Mitesh Patel By:  Judson Hannigan Name: Mitesh Patel Name: Judson Hannigan Title: VP, Marketing Title: CEO Newegg Inc. Allied Esports International, Inc. """

In [0]:
result = obfuscation_model.transform(spark.createDataFrame([[text]]).toDF("text"))

result.select(F.explode(F.arrays_zip(result.sentence.result, result.deidentified.result)).alias("cols")) \
      .select(F.expr("cols['0']").alias("sentence"), F.expr("cols['1']").alias("deidentified")).toPandas()

Out[32]:

,sentence,deidentified
0,"""Newegg"" ""Allied"" Newegg Inc.","""Newegg"" ""Allied"" Clark llc.."
1,"Allied Esports International, Inc.",SESA CO..
2,By Mitesh Patel By: Judson Hannigan Name: Mi...,By Benjamin Dean By: JAMES TURNER Name: Benj...
3,"Allied Esports International, Inc.",SESA CO..


## Use full pipeline in the Light model

In [0]:
light_model = nlp.LightPipeline(model)
annotated_text = light_model.annotate(text)
annotated_text['deidentified']

Out[33]: ['"<PARTY>" "<PARTY>" <PARTY>.',
 '<PARTY>.',
 'By <SIGNING_PERSON> By: <SIGNING_PERSON> Name: <SIGNING_PERSON> Name: <SIGNING_PERSON> Title: <SIGNING_TITLE> Title: <SIGNING_TITLE> <PARTY>.',
 '<PARTY>.']

In [0]:
obf_light_model = nlp.LightPipeline(obfuscation_model)
annotated_text = obf_light_model.annotate(text)
annotated_text['deidentified']

Out[34]: ['"Newegg" "Allied" Clark llc..',
 'SESA CO..',
 'By Benjamin Dean By: JAMES TURNER Name: Benjamin Dean Name: JAMES TURNER Title: Officer Title: Sales Manager Clark llc..',
 'SESA CO..']

# Save the Pipeline and Use it from Your Local

In [0]:
model.write().overwrite().save('/dbfs/pipeline_deid')

In [0]:
# from sparknlp.pretrained import PretrainedPipeline

deid_pipeline = nlp.PretrainedPipeline.from_disk("/dbfs/pipeline_deid")

In [0]:
data = spark.createDataFrame([[text]]).toDF("text")

In [0]:
deid_pipeline.model.stages

Out[39]: [DocumentAssembler_ed2b1a2c8f2e,
 SentenceDetector_834b6a5d2a80,
 REGEX_TOKENIZER_873a3034ace5,
 ROBERTA_EMBEDDINGS_b915dff90901,
 MedicalNerModel_93f728ff96e5,
 NER_CONVERTER_e9718cddf0b3,
 MedicalNerModel_2b2f0f671f99,
 NerConverter_449bfed38bd7,
 MERGE_fbaff3c3ed50,
 DE-IDENTIFICATION_98addba18cc4]

In [0]:
deid_pipeline.model.transform(data).show()

--------------------------------------------------------------------------- 
 IllegalArgumentException Traceback (most recent call last)
 <command-3850215879365759> in <module> 
 ----> 1 deid_pipeline . model . transform ( data ) . show ( ) 

 /databricks/python_shell/dbruntime/MLWorkloadsInstrumentation/_pyspark.py in patched_method (self, *args, **kwargs) 
 28 call_succeeded = False 
 29 try : 
 ---> 30 result = original_method ( self , * args , ** kwargs ) 
 31 call_succeeded = True 
 32 return result

 /databricks/spark/python/pyspark/ml/base.py in transform (self, dataset, params) 
 215 return self . copy ( params ) . _transform ( dataset ) 
 216 else : 
 --> 217 return self . _transform ( dataset ) 
 218 else : 
 219 raise ValueError ( "Params must be a param map but got %s." % type ( params ) ) 

 /databricks/spark/python/pyspark/ml/pipeline.py in _transform (self, dataset) 
 276 def _transform ( self , dataset ) : 
 277 for t in self . stages : 
 --> 278 dataset = t . transform ( dataset ) 
 279 return dataset
 280 

 /databricks/python_shell/dbruntime/MLWorkloadsInstrumentation/_pyspark.py in patched_method (self, *args, **kwargs) 
 28 call_succeeded = False 
 29 try : 
 ---> 30 result = original_method ( self , * args , ** kwargs ) 
 31 call_succeeded = True 
 32 return result

 /databricks/spark/python/pyspark/ml/base.py in transform (self, dataset, params) 
 215 return self . copy ( params ) . _transform ( dataset ) 
 216 else : 
 --> 217 return self . _transform ( dataset ) 
 218 else : 
 219 raise ValueError ( "Params must be a param map but got %s." % type ( params ) ) 

 /databricks/spark/python/pyspark/ml/wrapper.py in _transform (self, dataset) 
 352 def _transform ( self , dataset ) : 
 353 self . _transfer_params_to_java ( ) 
 --> 354 return DataFrame ( self . _java_obj . transform ( dataset . _jdf ) , dataset . sql_ctx ) 
 355 
 356 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 IllegalArgumentException : requirement failed: Wrong or missing inputCols annotators in MedicalNerModel_2b2f0f671f99.

Current inputCols: sentence,token,embeddings. Dataset's columns:
(column_name=text,is_nlp_annotator=false)
(column_name=document,is_nlp_annotator=true,type=document)
(column_name=sentence,is_nlp_annotator=true,type=document)
(column_name=token,is_nlp_annotator=true,type=token)
(column_name=embeddings_ner,is_nlp_annotator=true,type=word_embeddings)
(column_name=ner,is_nlp_annotator=true,type=named_entity)
(column_name=ner_chunk,is_nlp_annotator=true,type=chunk).
Make sure such annotators exist in your pipeline, with the right output names and that they have following annotator types: document, token, word_embeddings

# Pretrained Deidentification Pipeline

We have this pipeline can be used to deidentify legal information from texts.The legal information will be masked and obfuscated in the resulting text. The pipeline can mask and obfuscate `DOC`, `EFFDATE`, `PARTY`, `ALIAS`, `SIGNING_PERSON`, `SIGNING_TITLE`, `COUNTRY`, `CITY`, `STATE`, `STREET`, `ZIP`, `EMAIL`, `FAX`, `LOCATION-OTHER`, `DATE`,`PHONE` entities.

In [0]:
# from sparknlp.pretrained import PretrainedPipeline

deid_pipeline = nlp.PretrainedPipeline("legpipe_deid", "en", "legal/models")

legpipe_deid download started this may take some time.
Approx size to download 893.3 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][OK!]

In [0]:
deid_pipeline.model.stages

Out[43]: [DocumentAssembler_57ba7ce8bff9,
 SentenceDetectorDLModel_8aaebf7e098e,
 REGEX_TOKENIZER_2f265bb3f6b5,
 ROBERTA_EMBEDDINGS_b915dff90901,
 BERT_EMBEDDINGS_29ce72cd673e,
 LegalNerModel_f714c7246b46,
 NerConverter_5a5bb98a24c7,
 LegalNerModel_2b2f0f671f99,
 NerConverter_8b80797c7f67,
 LegalNerModel_7b3b98b32784,
 NER_CONVERTER_fb28b23bc35d,
 LegalNerModel_419e708135cb,
 NER_CONVERTER_af60235365b4,
 CONTEXTUAL-PARSER_85a13a5ff4bd,
 CONTEXTUAL-PARSER_bf8f02fb6658,
 REGEX_MATCHER_6199c32417bc,
 REGEX_MATCHER_2d694c8416b8,
 MERGE_5b96d578aa9b,
 DE-IDENTIFICATION_3d3dd57f734a,
 DE-IDENTIFICATION_471d94c72cd0,
 DE-IDENTIFICATION_29cac8c6cf56,
 DE-IDENTIFICATION_407b57c7d657,
 Finisher_ed29d709e530]

In [0]:
text= """CARGILL, INCORPORATED

By:     Pirkko Suominen



Name: Pirkko Suominen Title: Director, Bio Technology Development  Center,  Date:   10/19/2011

BIOAMBER, SAS

By:     Jean-François Huc



Name: Jean-François Huc  Title: President Date:   October 15, 2011

email : jeanfran@gmail.com
phone : 18087339090 """

In [0]:
deid_res= deid_pipeline.annotate(text)

In [0]:
deid_res.keys()

Out[46]: dict_keys(['obfuscated', 'deidentified', 'masked_fixed_length_chars', 'deid_merged_chunk', 'sentence', 'masked_with_chars'])

In [0]:
import pandas as pd
pd.set_option("display.max_colwidth", 100)

df= pd.DataFrame(list(zip(deid_res["sentence"], 
                          deid_res["deidentified"],
                          deid_res["masked_with_chars"],
                          deid_res["masked_fixed_length_chars"], 
                          deid_res["obfuscated"])),
                 columns= ["Sentence", "Masked", "Masked with Chars", "Masked with Fixed Chars", "Obfuscated"])

df

Out[47]:

,Sentence,Masked,Masked with Chars,Masked with Fixed Chars,Obfuscated
0,"CARGILL, INCORPORATED",<PARTY>,[*******************],****,TURER INC
1,By: Pirkko Suominen,By: <PARTY>,By: [*************],By: ****,By: SESA CO.
2,"Name: Pirkko Suominen Title: Director, Bio Technology Development Center, Date: 10/19/2011","Name: <PARTY>: <SIGNING_TITLE> Center, Date: <EFFDATE>","Name: [*******************]: [**********************************] Center, Date: [********]","Name: ****: **** Center, Date: ****","Name: John Snow Labs Inc: Sales Manager Center, Date: 03/08/2025"
3,"BIOAMBER, SAS","<PARTY>, <PARTY>","[******], [*]","****, ****","Clarus llc., SESA CO."
4,By: Jean-François Huc,By: <SIGNING_PERSON>,By: [***************],By: ****,By: JAMES TURNER
5,"Name: Jean-François Huc Title: President Date: October 15, 2011\n\nemail : jeanfran@gmail.com...",Name: <PARTY>: <SIGNING_TITLE>Date: <EFFDATE>\n\nemail : <EMAIL>\nphone : <PHONE>0,Name: [**********************]: [*******]Date: [**************]\n\nemail : [****************]\...,Name: ****: ****Date: ****\n\nemail : ****\nphone : ****0,"Name: MGT Trust Company, LLC.: Business ManagerDate: 11/7/2016\n\nemail : Berneta@hotmail.com\..."
